In [ ]:
# -*- coding: utf-8 -*-
"""
Colab 환경용 통합 실행 스크립트 (셀 1개 실행용)

개선된 SapBERT 기반 KCD → SNOMED 매핑 시스템
- Google Drive 마운트 + 경로 설정
- SapBERT 임베딩 + (옵션) FAISS 고속 검색
- 하이브리드 검색 (Lexical + Semantic)
- Re-ranking 전략
- Accuracy@1,5,10 / Top-K / MRR / NDCG@5,10 / MAP@10 / Avg Rank
- 시각화 + 결과/지표 저장
"""

# ================================
# 0. 기본 설정 및 드라이브 마운트
# ================================
import os
import gc
import re
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

import torch
print("CUDA available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda if torch.cuda.is_available() else "N/A")

# Colab 전용: Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

# ================================
# 1. 경로 설정 (필요시 수정)
# ================================
BASE_DIR   = "/content/drive/MyDrive/misoProject/base"
SOURCE_DIR = os.path.join(BASE_DIR, "source_data")
TARGET_DIR = os.path.join(BASE_DIR, "target_data")
TEST_DIR   = os.path.join(BASE_DIR, "test")
CACHE_DIR  = os.path.join(BASE_DIR, "cache")

os.makedirs(CACHE_DIR, exist_ok=True)
os.chdir(BASE_DIR)

print("📂 BASE_DIR:", BASE_DIR)
print("📂 내용:", os.listdir("."))
print("📂 source_data:", os.listdir(SOURCE_DIR))
print("📂 target_data:", os.listdir(TARGET_DIR))
print("📂 test:", os.listdir(TEST_DIR))

# ================================
# 2. 전처리 함수 정의
# (preprocessing_ver2_module.ipynb 내용을 여기로 옮겼다는 가정)
# 필요에 따라 실제 구현으로 교체
# ================================
import unicodedata
import string

def normalize_text(text: str) -> str:
    if not isinstance(text, str):
        text = str(text)
    # 예시: 소문자화 + NFKC 정규화 + 기호 제거 등
    text = unicodedata.normalize("NFKC", text)
    text = text.lower()
    # 숫자/영문/공백만 남기기 (필요시 수정)
    allowed = string.ascii_lowercase + string.digits + " "
    text = "".join(ch if ch in allowed else " " for ch in text)
    # 다중 공백 정리
    text = " ".join(text.split())
    return text

# 불용어 제거 (영문 간단 예시, 실제 리스트는 확장 가능)
STOPWORDS = set(["of", "the", "and", "a", "an", "in", "on", "with", "for", "to"])

def remove_stopwords(text: str) -> str:
    tokens = text.split()
    tokens = [t for t in tokens if t not in STOPWORDS]
    return " ".join(tokens)

# 단순 복수형 정규화 (예시)
def normalize_plural(text: str) -> str:
    tokens = text.split()
    norm_tokens = []
    for t in tokens:
        if t.endswith("s") and len(t) > 3:
            norm_tokens.append(t[:-1])
        else:
            norm_tokens.append(t)
    return " ".join(norm_tokens)

def apply_morph(text: str) -> str:
    """간단 형태소/토큰 분석기 (현재는 토큰화 수준, 필요시 Mecab 등 연동)"""
    if not isinstance(text, str):
        text = str(text)
    x = normalize_text(text)
    tokens = x.split()
    return " ".join(tokens)

print("✅ 전처리 함수 정의 완료")
print("예시 normalize_text('Test TEXT!'):", normalize_text("Test TEXT!"))


# 형태소 분석 사용 여부
USE_MORPH_FOR_KCD    = False
USE_MORPH_FOR_SNOMED = False

print(f"USE_MORPH_FOR_KCD    = {USE_MORPH_FOR_KCD}")
print(f"USE_MORPH_FOR_SNOMED = {USE_MORPH_FOR_SNOMED}")


# ================================
# 3. SNOMED TSV 안전 로더
# ================================
def load_snomed_tsv(path: str, usecols=None) -> pd.DataFrame:
    encodings = [
        "utf-8", "utf-8-sig",
        "utf-16", "utf-16le", "utf-16be",
        "latin1", "cp1252", "ISO-8859-1",
    ]
    last_err = None
    for enc in encodings:
        try:
            df = pd.read_csv(
                path,
                sep="\t",
                dtype=str,
                low_memory=False,
                encoding=enc,
                usecols=usecols
            )
            print(f"✅ {os.path.basename(path)}: decoded with {enc}")
            return df
        except Exception as e:
            last_err = e
    raise RuntimeError(f"❗ Failed to decode {path}. Last error: {last_err}")


# ================================
# 4. SNOMED 파일 경로 감지 및 로드
# ================================
concept_file = [f for f in os.listdir(TARGET_DIR) if "Concept_Snapshot" in f][0]
desc_file    = [f for f in os.listdir(TARGET_DIR) if "Description_Snapshot" in f][0]
rel_file     = [f for f in os.listdir(TARGET_DIR) if "Relationship_Snapshot" in f][0]

concept_path = os.path.join(TARGET_DIR, concept_file)
desc_path    = os.path.join(TARGET_DIR, desc_file)
rel_path     = os.path.join(TARGET_DIR, rel_file)

print("Concept  :", concept_path)
print("Desc     :", desc_path)
print("Relation :", rel_path)

concept_cols = ["id", "active"]
df_concept = load_snomed_tsv(concept_path, usecols=concept_cols)
print("Concept raw shape:", df_concept.shape)

desc_cols = ["id", "active", "languageCode", "conceptId", "typeId", "term"]
df_desc = load_snomed_tsv(desc_path, usecols=desc_cols)
print("Desc raw shape:", df_desc.shape)

rel_cols = ["active", "sourceId", "destinationId", "typeId"]
df_rel = load_snomed_tsv(rel_path, usecols=rel_cols)
print("Rel raw shape:", df_rel.shape)


# ================================
# 5. Concept/Description/Relation 전처리
# ================================
df_concept = df_concept[df_concept["active"] == "1"][["id"]].copy()
df_concept = df_concept.rename(columns={"id": "conceptId"})
print("Concept active=1:", len(df_concept))

df_desc = df_desc[
    (df_desc["active"] == "1") &
    (df_desc["languageCode"] == "en")
][["conceptId", "typeId", "term"]].copy()
print("Desc active=1 & en:", len(df_desc))

FSN_ID = "900000000000003001"
SYN_ID = "900000000000013009"

def extract_semantic_tag(term: str) -> str:
    m = re.search(r"\(([^()]*)\)\s*$", str(term))
    return m.group(1).lower() if m else ""

df_fsn = df_desc[df_desc["typeId"] == FSN_ID][["conceptId", "term"]].copy()
df_fsn["semantic_tag"] = df_fsn["term"].apply(extract_semantic_tag)

df_fsn_disorder = df_fsn[df_fsn["semantic_tag"] == "disorder"].copy()
disorder_ids = set(df_fsn_disorder["conceptId"])
print("disorder concept 개수:", len(disorder_ids))

del df_fsn, df_fsn_disorder
gc.collect()

df_desc = df_desc[df_desc["conceptId"].isin(disorder_ids)].copy()
print("Desc (disorder only):", len(df_desc))

df_rel = df_rel[df_rel["active"] == "1"][["sourceId", "destinationId", "typeId"]].copy()
print("Rel active=1:", len(df_rel))

IS_A = "116680003"
df_rel_isa = df_rel[df_rel["typeId"] == IS_A][["sourceId", "destinationId"]].copy()
print("is-a 관계 행수:", len(df_rel_isa))

del df_rel
gc.collect()

df_parent = (
    df_rel_isa
    .groupby("sourceId")["destinationId"]
    .apply(list)
    .reset_index()
    .rename(columns={"sourceId": "conceptId", "destinationId": "parent_concepts"})
)

df_child = (
    df_rel_isa
    .groupby("destinationId")["sourceId"]
    .apply(list)
    .reset_index()
    .rename(columns={"destinationId": "conceptId", "sourceId": "child_concepts"})
)

print("parent 리스트 행수:", len(df_parent))
print("child  리스트 행수:", len(df_child))

del df_rel_isa
gc.collect()

def map_term_type(tid: str) -> str:
    if tid == FSN_ID:
        return "FSN"
    elif tid == SYN_ID:
        return "SYN"
    else:
        return "OTHER"

df_desc["term_type"] = df_desc["typeId"].apply(map_term_type)

def normalize_for_snomed(text: str) -> str:
    x = normalize_text(text)
    if USE_MORPH_FOR_SNOMED:
        x = apply_morph(x)
    x = remove_stopwords(x)
    x = normalize_plural(x)
    return x

df_desc["normalized_term"] = df_desc["term"].apply(normalize_for_snomed)

print("df_desc normalized_term 예시:")
print(df_desc[["term", "term_type", "normalized_term"]].head())

df_snomed = df_desc.merge(
    df_concept[["conceptId"]],
    on="conceptId",
    how="inner"
)

print("Concept + Desc 조합 행수:", len(df_snomed))

del df_concept, df_desc
gc.collect()

df_snomed = df_snomed.merge(df_parent, on="conceptId", how="left")
df_snomed = df_snomed.merge(df_child, on="conceptId", how="left")

del df_parent, df_child
gc.collect()

dedup_cols = ["conceptId", "term", "term_type", "normalized_term"]
snomed_index = (
    df_snomed
    .drop_duplicates(subset=dedup_cols)
    .reset_index(drop=True)
    [["conceptId", "term", "term_type", "normalized_term",
      "parent_concepts", "child_concepts"]]
)

del df_snomed
gc.collect()

print("✅ 최종 snomed_index 행수:", len(snomed_index))
print(snomed_index.head(5))


# ================================
# 6. TEST 데이터 로드 및 전처리
# ================================
test_file = os.path.join(TEST_DIR, "test_source.xlsx")
df_test = pd.read_excel(test_file, dtype=str).reset_index(drop=True)
print("✔ 테스트 데이터 로드:", df_test.shape)
print(df_test.head())

if "source_code" in df_test.columns:
    df_test = df_test.rename(columns={"source_code": "kcd_code"})
if "source_en_name" in df_test.columns:
    df_test = df_test.rename(columns={"source_en_name": "kcd_en"})

df_test["kcd_en"] = df_test["kcd_en"].fillna("")

def normalize_for_kcd(text: str) -> str:
    x = normalize_text(text)
    if USE_MORPH_FOR_KCD:
        x = apply_morph(x)
    x = remove_stopwords(x)
    x = normalize_plural(x)
    return x

df_test["kcd_normalized"] = df_test["kcd_en"].apply(normalize_for_kcd)
df_test["kcd_no_sw"]      = df_test["kcd_normalized"]
df_test["kcd_no_plural"]  = df_test["kcd_normalized"].apply(normalize_plural)

print("✔ 전처리 예시:")
print(df_test[["kcd_en","kcd_normalized","kcd_no_sw","kcd_no_plural"]].head())


# ================================
# 7. SapBERT 로드 + SNOMED 임베딩
# ================================
from sentence_transformers import SentenceTransformer

device = "cuda" if torch.cuda.is_available() else "cpu"
print("🔥 Using device:", device)

sapbert = SentenceTransformer(
    "cambridgeltl/SapBERT-UMLS-2020AB-all-lang-from-XLMR",
    device=device
)

def encode_in_batches(model, texts, batch_size=32, cache_path=None):
    """대용량 텍스트를 배치로 나눠 인코딩하고 캐싱"""
    if cache_path and os.path.exists(cache_path):
        print(f"✅ 캐시에서 로드: {cache_path}")
        return np.load(cache_path)

    embeddings = []
    total_batches = (len(texts) + batch_size - 1) // batch_size

    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        batch_emb = model.encode(
            batch,
            convert_to_numpy=True,
            normalize_embeddings=True,
            show_progress_bar=False
        )
        embeddings.append(batch_emb)

        if (i // batch_size) % 100 == 0:
            print(f"  진행: {i // batch_size + 1}/{total_batches} 배치")

    embeddings = np.vstack(embeddings).astype("float32")

    if cache_path:
        np.save(cache_path, embeddings)
        print(f"💾 캐시 저장 완료: {cache_path}")

    return embeddings

SAPBERT_CACHE = os.path.join(CACHE_DIR, "snomed_sapbert_embs_v2.npy")
snomed_terms = snomed_index["normalized_term"].fillna("").tolist()

print("🔄 SNOMED 임베딩 생성 중...")
snomed_embs = encode_in_batches(
    sapbert,
    snomed_terms,
    batch_size=32,
    cache_path=SAPBERT_CACHE
)
print(f"✅ SNOMED 임베딩 완료: {snomed_embs.shape}")


# ================================
# 8. (옵션) FAISS 설정
# ================================
try:
    import faiss
    FAISS_AVAILABLE = True
    print("✅ FAISS 사용 가능")
except ImportError:
    FAISS_AVAILABLE = False
    print("⚠️ FAISS 미설치 - sklearn 사용")

def build_faiss_index(embeddings, use_gpu=True):
    d = embeddings.shape[1]
    index = faiss.IndexFlatIP(d)
    if use_gpu and faiss.get_num_gpus() > 0:
        res = faiss.StandardGpuResources()
        index = faiss.index_cpu_to_gpu(res, 0, index)
        print("🔥 GPU FAISS 사용")
    else:
        print("💻 CPU FAISS 사용")
    index.add(embeddings)
    return index

def faiss_topk_search(index, query_embs, k=5):
    scores, indices = index.search(query_embs, k)
    return indices, scores

def sklearn_topk_search(query_embs, snomed_embs, k=5, chunk_size=20000):
    N = len(query_embs)
    M = len(snomed_embs)
    all_scores = np.full((N, k), -9999, dtype="float32")
    all_idx    = np.full((N, k), -1,    dtype="int32")

    for start in range(0, M, chunk_size):
        end = min(start+chunk_size, M)
        snomed_chunk = snomed_embs[start:end]
        sims = cosine_similarity(query_embs, snomed_chunk)

        local_topk = np.argpartition(-sims, k-1, axis=1)[:, :k]
        local_scores = np.take_along_axis(sims, local_topk, axis=1)
        global_idx = local_topk + start

        comb_scores = np.concatenate([all_scores, local_scores], axis=1)
        comb_idx    = np.concatenate([all_idx, global_idx], axis=1)

        new_topk = np.argpartition(-comb_scores, k-1, axis=1)[:, :k]
        all_scores = np.take_along_axis(comb_scores, new_topk, axis=1)
        all_idx    = np.take_along_axis(comb_idx,    new_topk, axis=1)

    return all_idx, all_scores


# ================================
# 9. 하이브리드 검색 + Re-ranking
# (메모리 문제 방지 위해 "semantic-only 모드"를 기본으로)
# ================================
USE_HYBRID = False      # Colab에서 메모리 안전 위해 기본 False
HYBRID_ALPHA = 0.7
USE_RERANK = True
TOPK = 10

def hybrid_search(query_texts, snomed_texts, snomed_embs, sapbert_model,
                  alpha=0.7, k=10, use_faiss=True):
    """
    (주의) 전체 코퍼스에 대해 full matrix를 만들기 때문에,
    SNOMED 규모가 크면 Colab에서 메모리 터질 수 있음.
    """
    print(f"🔍 하이브리드 검색 시작 (alpha={alpha}, k={k})")

    print("  📊 TF-IDF 계산 중...")
    tfidf = TfidfVectorizer(max_features=5000, min_df=2)
    snomed_tfidf = tfidf.fit_transform(snomed_texts)
    query_tfidf = tfidf.transform(query_texts)

    # full lexical similarity (메모리 주의)
    lexical_scores = cosine_similarity(query_tfidf, snomed_tfidf)

    print("  🧠 SapBERT 임베딩 중...")
    query_embs = sapbert_model.encode(
        query_texts,
        batch_size=16,
        convert_to_numpy=True,
        normalize_embeddings=True,
        show_progress_bar=True
    )

    if use_faiss and FAISS_AVAILABLE:
        print("  ⚡ FAISS 검색...")
        faiss_index = build_faiss_index(snomed_embs, use_gpu=(device == "cuda"))
        semantic_scores = cosine_similarity(query_embs, snomed_embs)
    else:
        print("  💻 sklearn 검색...")
        semantic_scores = cosine_similarity(query_embs, snomed_embs)

    print("  🔀 하이브리드 점수 계산...")
    hybrid_scores = alpha * semantic_scores + (1 - alpha) * lexical_scores

    topk_idx = np.argsort(-hybrid_scores, axis=1)[:, :k]
    topk_scores = np.take_along_axis(hybrid_scores, topk_idx, axis=1)

    print(f"✅ 하이브리드 검색 완료")
    return topk_idx, topk_scores

def rerank_with_heuristics(topk_idx, topk_scores, df_test, snomed_index):
    print("🎯 Re-ranking 시작...")
    reranked_idx = []
    reranked_scores = []

    for i in range(len(df_test)):
        cand_indices = topk_idx[i]
        cand_scores = topk_scores[i].copy()
        query_len = len(df_test.iloc[i]["kcd_normalized"].split())

        for j, idx in enumerate(cand_indices):
            info = snomed_index.iloc[idx]

            if info["term_type"] == "FSN":
                cand_scores[j] += 0.05

            cand_len = len(str(info["normalized_term"]).split())
            if query_len > 0 and cand_len > 0:
                len_sim = 1 - abs(query_len - cand_len) / max(query_len, cand_len)
                cand_scores[j] += 0.03 * len_sim

        sorted_order = np.argsort(-cand_scores)
        reranked_idx.append(cand_indices[sorted_order])
        reranked_scores.append(cand_scores[sorted_order])

    print("✅ Re-ranking 완료")
    return np.array(reranked_idx), np.array(reranked_scores)


# ================================
# 10. 검색 실행 (기본: SapBERT-only)
# ================================
if USE_HYBRID:
    print(f"\n{'='*60}")
    print("🔍 하이브리드 검색 모드")
    print(f"{'='*60}\n")

    topk_idx, topk_scores = hybrid_search(
        df_test["kcd_normalized"].tolist(),
        snomed_index["normalized_term"].tolist(),
        snomed_embs,
        sapbert,
        alpha=HYBRID_ALPHA,
        k=TOPK,
        use_faiss=FAISS_AVAILABLE
    )
else:
    print(f"\n{'='*60}")
    print("🧠 순수 SapBERT 검색 모드")
    print(f"{'='*60}\n")

    query_texts = df_test["kcd_no_plural"].tolist()
    query_embs = sapbert.encode(
        query_texts,
        batch_size=16,
        convert_to_numpy=True,
        show_progress_bar=True,
        normalize_embeddings=True
    ).astype("float32")

    if FAISS_AVAILABLE:
        print("⚡ FAISS 검색 중...")
        faiss_index = build_faiss_index(snomed_embs, use_gpu=(device == "cuda"))
        topk_idx, topk_scores = faiss_topk_search(faiss_index, query_embs, k=TOPK)
    else:
        print("💻 sklearn 검색 중...")
        topk_idx, topk_scores = sklearn_topk_search(query_embs, snomed_embs, k=TOPK)

if USE_RERANK:
    topk_idx, topk_scores = rerank_with_heuristics(
        topk_idx, topk_scores, df_test, snomed_index
    )


# ================================
# 11. 결과 생성
# ================================
conceptIds = snomed_index["conceptId"].tolist()
terms      = snomed_index["term"].tolist()

df_pred = df_test.copy()
df_pred["sapbert_best_idx"] = [idxs[0] for idxs in topk_idx]
df_pred["pred_conceptId"]   = df_pred["sapbert_best_idx"].apply(lambda x: conceptIds[x])
df_pred["pred_term"]        = df_pred["sapbert_best_idx"].apply(lambda x: terms[x])
df_pred["pred_score"]       = [scores[0] for scores in topk_scores]
df_pred["mapping_step"]     = "hybrid_rerank" if USE_HYBRID else "sapbert_only"

topk_pred_ids = []
topk_pred_terms = []
topk_pred_scores_list = []

for i in range(len(df_test)):
    idx_list = topk_idx[i]
    cand_ids = [str(conceptIds[idx]) for idx in idx_list]
    cand_terms = [str(terms[idx]) for idx in idx_list]
    cand_scores = [float(s) for s in topk_scores[i]]

    topk_pred_ids.append(cand_ids)
    topk_pred_terms.append(cand_terms)
    topk_pred_scores_list.append(cand_scores)

df_pred[f"top{TOPK}_conceptIds"] = topk_pred_ids
df_pred[f"top{TOPK}_terms"]      = topk_pred_terms
df_pred[f"top{TOPK}_scores"]     = topk_pred_scores_list


# ================================
# 12. 정답 로드 + Accuracy/Top-K/MRR/NDCG/MAP/AvgRank
# ================================
target_file = os.path.join(TEST_DIR, "test_target_with_source_code.xlsx")
df_target = pd.read_excel(target_file, dtype=str).fillna("")

df_target = df_target.rename(columns={
    "concept_code": "true_conceptId",
    "concept_name": "true_term"
})

df_eval = df_pred.merge(df_target, left_index=True, right_index=True)

df_eval["correct"] = (
    df_eval["pred_conceptId"].astype(str) == df_eval["true_conceptId"].astype(str)
)
top1_accuracy = df_eval["correct"].mean() * 100

topk_correct = []
for i, row in df_eval.iterrows():
    true_id = str(row["true_conceptId"])
    cand_ids = row[f"top{TOPK}_conceptIds"]
    topk_correct.append(true_id in cand_ids)

df_eval["topk_correct"] = topk_correct
topk_accuracy = df_eval["topk_correct"].mean() * 100

ranks_rr = []
for i in range(len(df_test)):
    true_id = str(df_target.loc[i, "true_conceptId"])
    cand_ids = df_pred.loc[i, f"top{TOPK}_conceptIds"]
    rr = 0.0
    for rank, cid in enumerate(cand_ids, start=1):
        if cid == true_id:
            rr = 1.0 / rank
            break
    ranks_rr.append(rr)
MRR = sum(ranks_rr) / len(ranks_rr)

def dcg_at_k(relevances, k):
    relevances = np.asarray(relevances)[:k]
    if relevances.size == 0:
        return 0.0
    discounts = 1.0 / np.log2(np.arange(2, relevances.size + 2))
    return float(np.sum(relevances * discounts))

def ndcg_at_k_binary(relevances, k):
    dcg = dcg_at_k(relevances, k)
    ideal_rels = [1] + [0] * (k - 1)
    idcg = dcg_at_k(ideal_rels, k)
    if idcg == 0:
        return 0.0
    return dcg / idcg

def average_precision_at_k_binary(relevances, k):
    relevances = np.asarray(relevances)[:k]
    if np.sum(relevances) == 0:
        return 0.0
    precisions = []
    num_hits = 0
    for i, rel in enumerate(relevances, start=1):
        if rel > 0:
            num_hits += 1
            precisions.append(num_hits / i)
    if len(precisions) == 0:
        return 0.0
    return float(np.mean(precisions))

K_MAX = TOPK
acc_list = []
for k in range(1, TOPK + 1):
    correct_at_k = 0
    for i in range(len(df_test)):
        true_id = str(df_target.loc[i, "true_conceptId"])
        cand_ids = df_pred.loc[i, f"top{TOPK}_conceptIds"][:k]
        if true_id in cand_ids:
            correct_at_k += 1
    acc_list.append(correct_at_k / len(df_test))

top1_acc = acc_list[0]
top5_acc = acc_list[min(4, K_MAX - 1)]
top10_acc = acc_list[min(9, K_MAX - 1)]

ndcg5_list = []
ndcg10_list = []
ap10_list = []
avg_rank_list = []

for i in range(len(df_test)):
    true_id = str(df_target.loc[i, "true_conceptId"])
    cand_ids_full = df_pred.loc[i, f"top{TOPK}_conceptIds"]
    relevances = [1 if cid == true_id else 0 for cid in cand_ids_full]

    ndcg5  = ndcg_at_k_binary(relevances, k=min(5, K_MAX))
    ndcg10 = ndcg_at_k_binary(relevances, k=min(10, K_MAX))
    ndcg5_list.append(ndcg5)
    ndcg10_list.append(ndcg10)

    ap10 = average_precision_at_k_binary(relevances, k=min(10, K_MAX))
    ap10_list.append(ap10)

    rank = 0
    for r, cid in enumerate(cand_ids_full, start=1):
        if cid == true_id:
            rank = r
            break
    if rank == 0:
        rank = K_MAX + 1
    avg_rank_list.append(rank)

ndcg5_mean = float(np.mean(ndcg5_list))
ndcg10_mean = float(np.mean(ndcg10_list))
map10_mean = float(np.mean(ap10_list))
avg_rank = float(np.mean(avg_rank_list))

print(f"\n{'='*60}")
print("📊 최종 성능 평가 결과")
print(f"{'='*60}")
print(f"🎯 Top-1 Accuracy: {top1_accuracy:.2f}%")
print(f"🎯 Top-{TOPK} Accuracy: {topk_accuracy:.2f}%")
print(f"🎯 MRR (Mean Reciprocal Rank): {MRR:.4f}")
print(f"🎯 Accuracy@5: {top5_acc*100:.2f}%")
print(f"🎯 Accuracy@10: {top10_acc*100:.2f}%")
print(f"🎯 NDCG@5: {ndcg5_mean:.4f}")
print(f"🎯 NDCG@10: {ndcg10_mean:.4f}")
print(f"🎯 MAP@10: {map10_mean:.4f}")
print(f"🎯 Average Rank: {avg_rank:.2f}")
print(f"{'='*60}\n")


# ================================
# 13. 오류 분석
# ================================
def analyze_errors(df_eval, snomed_index, top_n=10):
    errors = df_eval[~df_eval["correct"]].copy()

    print(f"\n{'='*60}")
    print(f"❌ 오류 분석")
    print(f"{'='*60}")
    print(f"총 오류 개수: {len(errors)} / {len(df_eval)}")
    print(f"정확도: {df_eval['correct'].mean() * 100:.2f}%\n")

    if len(errors) == 0:
        print("🎉 오류 없음!")
        return errors

    print(f"🔍 High-Confidence 오류 (Top {min(top_n, len(errors))}):")
    confident_errors = errors.nlargest(min(top_n, len(errors)), "pred_score")
    for idx, row in confident_errors.iterrows():
        print(f"\n  [{row['kcd_code']}] {row['kcd_en']}")
        print(f"    예측: {row['pred_conceptId']} - {row['pred_term']}")
        print(f"    점수: {row['pred_score']:.3f}")
        print(f"    정답: {row['true_conceptId']} - {row['true_term']}")

    print(f"\n\n🤔 Low-Confidence 오류 (Bottom {min(top_n, len(errors))}):")
    uncertain = errors.nsmallest(min(top_n, len(errors)), "pred_score")
    for idx, row in uncertain.head(5).iterrows():
        print(f"\n  [{row['kcd_code']}] {row['kcd_en']}")
        print(f"    예측 점수: {row['pred_score']:.3f}")
        print(f"    정답: {row['true_term']}")

    print(f"\n\n📊 오류 케이스 점수 통계:")
    print(f"  평균: {errors['pred_score'].mean():.3f}")
    print(f"  중앙값: {errors['pred_score'].median():.3f}")
    print(f"  최소: {errors['pred_score'].min():.3f}")
    print(f"  최대: {errors['pred_score'].max():.3f}")
    print(f"{'='*60}\n")
    return errors

errors_df = analyze_errors(df_eval, snomed_index, top_n=10)


# ================================
# 14. 시각화
# ================================
import matplotlib.pyplot as plt

rank_list = []
for i in range(len(df_test)):
    true_id = str(df_target.loc[i, "true_conceptId"])
    cand_ids = df_pred.loc[i, f"top{TOPK}_conceptIds"]
    rank = 0
    for r, cid in enumerate(cand_ids, start=1):
        if cid == true_id:
            rank = r
            break
    rank_list.append(rank)

rank_counts = {}
for r in rank_list:
    rank_counts[r] = rank_counts.get(r, 0) + 1

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

ax0 = axes[0, 0]
ax0.plot(range(1, TOPK + 1), [a * 100 for a in acc_list], marker='o', linewidth=2)
ax0.set_xlabel('K', fontsize=12)
ax0.set_ylabel('Accuracy (%)', fontsize=12)
ax0.set_title('Accuracy@K Curve', fontsize=14, fontweight='bold')
ax0.grid(True, alpha=0.3)
ax0.set_xticks(range(1, TOPK + 1))

ax1 = axes[0, 1]
ranks = sorted([r for r in rank_counts.keys() if r != 0])
counts = [rank_counts[r] for r in ranks]
ax1.bar(ranks, counts, color='steelblue', alpha=0.7)
ax1.set_xlabel('Rank of Correct Answer', fontsize=12)
ax1.set_ylabel('Number of Samples', fontsize=12)
ax1.set_title('Distribution of Correct Answer Rank', fontsize=14, fontweight='bold')
ax1.set_xticks(ranks)

ax2 = axes[1, 0]
top1_scores = [float(scores[0]) for scores in topk_scores]
ax2.hist(top1_scores, bins=30, color='coral', alpha=0.7, edgecolor='black')
ax2.set_xlabel('Top-1 Similarity Score', fontsize=12)
ax2.set_ylabel('Number of Samples', fontsize=12)
ax2.set_title('Top-1 Score Distribution', fontsize=14, fontweight='bold')
ax2.axvline(np.mean(top1_scores), color='red', linestyle='--',
            linewidth=2, label=f'Mean: {np.mean(top1_scores):.3f}')
ax2.legend()

ax3 = axes[1, 1]
metric_names = ['Acc@1', 'Acc@5', 'Acc@10', 'MRR', 'NDCG@5', 'NDCG@10', 'MAP@10']
metric_vals = [
    top1_acc * 100,
    top5_acc * 100,
    top10_acc * 100,
    MRR * 100,
    ndcg5_mean * 100,
    ndcg10_mean * 100,
    map10_mean * 100
]
ax3.bar(metric_names, metric_vals, color='seagreen', alpha=0.8)
ax3.set_ylabel('Score (%)', fontsize=12)
ax3.set_title('Ranking Metrics Summary', fontsize=14, fontweight='bold')
ax3.set_ylim(0, 100)
for x, v in zip(metric_names, metric_vals):
    ax3.text(x, v + 1, f"{v:.1f}", ha='center', va='bottom', fontsize=9)

plt.tight_layout()
viz_path = os.path.join(TEST_DIR, "performance_visualization_extended.png")
plt.savefig(viz_path, dpi=300, bbox_inches='tight')
plt.show()
print(f"📁 확장 시각화 저장 완료: {viz_path}")


# ================================
# 15. 결과 저장
# ================================
output_main = os.path.join(TEST_DIR, "improved_sapbert_result.xlsx")
output_errors = os.path.join(TEST_DIR, "error_analysis.xlsx")
output_metrics = os.path.join(TEST_DIR, "performance_metrics.csv")
output_extra_metrics = os.path.join(TEST_DIR, "performance_metrics_extended.csv")

df_eval.to_excel(output_main, index=False)
print(f"📁 전체 결과 저장: {output_main}")

if len(errors_df) > 0:
    errors_df.to_excel(output_errors, index=False)
    print(f"📁 오류 분석 저장: {output_errors}")

metrics_df = pd.DataFrame({
    'K': list(range(1, TOPK + 1)),
    'Accuracy': [a * 100 for a in acc_list]
})
metrics_df.to_csv(output_metrics, index=False)
print(f"📁 성능 지표 (Accuracy@K) 저장: {output_metrics}")

extra_metrics = {
    'Top1_Accuracy': top1_acc * 100,
    'Top5_Accuracy': top5_acc * 100,
    'Top10_Accuracy': top10_acc * 100,
    'MRR': MRR,
    'NDCG@5': ndcg5_mean,
    'NDCG@10': ndcg10_mean,
    'MAP@10': map10_mean,
    'AvgRank': avg_rank
}
extra_metrics_df = pd.DataFrame([extra_metrics])
extra_metrics_df.to_csv(output_extra_metrics, index=False)
print(f"📁 확장 성능 지표 저장: {output_extra_metrics}")

summary_path = os.path.join(TEST_DIR, "performance_summary.txt")
with open(summary_path, 'w', encoding='utf-8') as f:
    f.write("="*60 + "\n")
    f.write("개선된 SapBERT 시스템 성능 리포트\n")
    f.write("="*60 + "\n\n")
    f.write(f"검색 모드: {'하이브리드 (Lexical + Semantic)' if USE_HYBRID else '순수 SapBERT'}\n")
    if USE_HYBRID:
        f.write(f"하이브리드 가중치 (alpha): {HYBRID_ALPHA}\n")
    f.write(f"Re-ranking 적용: {'예' if USE_RERANK else '아니오'}\n")
    f.write(f"FAISS 사용: {'예' if FAISS_AVAILABLE else '아니오 (sklearn)'}\n\n")
    f.write("-"*60 + "\n")
    f.write("핵심 성능 지표\n")
    f.write("-"*60 + "\n")
    f.write(f"Top-1 Accuracy: {top1_accuracy:.2f}%\n")
    f.write(f"Top-{TOPK} Accuracy: {topk_accuracy:.2f}%\n")
    f.write(f"MRR: {MRR:.4f}\n")
    f.write(f"Accuracy@5: {top5_acc*100:.2f}%\n")
    f.write(f"Accuracy@10: {top10_acc*100:.2f}%\n")
    f.write(f"NDCG@5: {ndcg5_mean:.4f}\n")
    f.write(f"NDCG@10: {ndcg10_mean:.4f}\n")
    f.write(f"MAP@10: {map10_mean:.4f}\n")
    f.write(f"Average Rank: {avg_rank:.2f}\n\n")
    f.write("-"*60 + "\n")
    f.write("Accuracy@K 상세\n")
    f.write("-"*60 + "\n")
    for k, acc in enumerate(acc_list, 1):
        f.write(f"Accuracy@{k}: {acc * 100:.2f}%\n")
    f.write("\n" + "-"*60 + "\n")
    f.write("Top-1 Score 통계\n")
    f.write("-"*60 + "\n")
    f.write(f"평균: {np.mean(top1_scores):.4f}\n")
    f.write(f"중앙값: {np.median(top1_scores):.4f}\n")
    f.write(f"표준편차: {np.std(top1_scores):.4f}\n")
    f.write(f"최소: {np.min(top1_scores):.4f}\n")
    f.write(f"최대: {np.max(top1_scores):.4f}\n")

print(f"📁 요약 리포트 저장: {summary_path}")

print(f"\n{'='*60}")
print("✅ 모든 작업 완료!")
print(f"{'='*60}")
print("저장된 파일:")
print(f"  - {output_main}")
print(f"  - {output_errors}")
print(f"  - {output_metrics}")
print(f"  - {output_extra_metrics}")
print(f"  - {summary_path}")
print(f"  - {viz_path}")
